In [1]:
## Multi- Spectrum Fitting by Segment

import numpy as np
import numpy.ma as ma
import pandas as pd
import qgrid
import os, sys
from lmfit import minimize, Parameters, report_fit, Model, Minimizer
from lmfit.models import GaussianModel, LinearModel, Model, VoigtModel
import matplotlib.pyplot as plt
#from matplotlib import gridspec
#from scipy import interpolate


sys.path.append(r'C:\Users\ema3\Documents\Python Scripts\HAPI')# set location of HAPI.py module
%matplotlib inline
#from hapi import *
from hapi_HTP_definition_dataframe_190409 import *


import seaborn as sns
sns.set_style("whitegrid")
sns.set_style("ticks")
sns.set_context("poster")


HAPI version: 1.1.0.9.6
To get the most up-to-date version please check http://hitran.org/hapi
ATTENTION: Python versions of partition sums from TIPS-2017 are now available in HAPI code

           It is free to use HAPI. If you use HAPI in your research or software development,
           please cite it using the following reference:
           R.V. Kochanov, I.E. Gordon, L.S. Rothman, P. Wcislo, C. Hill, J.S. Wilzewski,
           HITRAN Application Programming Interface (HAPI): A comprehensive approach
           to working with spectroscopic data, J. Quant. Spectrosc. Radiat. Transfer 177, 15-30 (2016)
           DOI: 10.1016/j.jqsrt.2016.03.005


In [2]:
path = r'C:\Users\ema3\Documents\Data\Helene Singlet Delta Data' # Location of the Summary Data File
os.chdir(path)

wave_range = 1.5 #range outside of experimental x-range to simulate
IntensityThreshold = 1e-30 #intensities must be above this value to be simulated
Fit_Intensity = 1e-27 #intensities must be above this value for the line to be fit
order_baseline_fit = 2
tau_column = 'Mean tau/us corr offset' 
freq_column = 'Real freq' 
temp_column = 'Temperature /C'
pressure_column = 'New Calculated Pressure /Torr'


spec_1 = Spectrum('25Torr-all-offsetcorr-26-goodPressure', 
                        concentration = { 7 : 0.2093}, natural_abundance = True, diluent = 'air', 
                        etalons = {}, 
                        input_freq = True, frequency_column = freq_column,
                        input_tau = True, tau_column = tau_column, tau_stats_column = None, 
                        pressure_column = pressure_column, temperature_column = temp_column, 
                        nominal_temperature = 296, abundance_ratio_MI = {}, x_shift = 0.00)
spec_2 = Spectrum('25Torr-all-offsetcorr-29-goodPressure', 
                        concentration = { 7 : 0.2093}, natural_abundance = True, diluent = 'air', 
                        etalons = {}, 
                        input_freq = True, frequency_column = freq_column,
                        input_tau = True, tau_column = tau_column, tau_stats_column = None, 
                        pressure_column = pressure_column, temperature_column = temp_column, 
                        nominal_temperature = 296, abundance_ratio_MI = {}, x_shift = 0.00)
spec_3 = Spectrum('25Torr-all-offsetcorr-2728-goodPressure', 
                        concentration = { 7 :0.2093}, natural_abundance = True, diluent = 'air', 
                        etalons = {}, 
                        input_freq = True, frequency_column = freq_column,
                        input_tau = True, tau_column = tau_column, tau_stats_column = None, 
                        pressure_column = pressure_column, temperature_column = temp_column, 
                        nominal_temperature = 296, abundance_ratio_MI = {}, x_shift = 0.00)
spec_4 = Spectrum('60Torr-all-offsetcorr-goodPressure', 
                        concentration = { 7 : 0.2093}, natural_abundance = True, diluent = 'air', 
                        etalons = {}, 
                        input_freq = True, frequency_column = freq_column,
                        input_tau = True, tau_column = tau_column, tau_stats_column = None, 
                        pressure_column = pressure_column, temperature_column = temp_column, 
                        nominal_temperature = 296, abundance_ratio_MI = {}, x_shift = 0.00)
spec_5 = Spectrum('110Torr-all-offsetcorr-goodPressure', 
                        concentration = { 7 : 0.2093}, natural_abundance = True, diluent = 'air', 
                        etalons = {}, 
                        input_freq = True, frequency_column = freq_column,
                        input_tau = True, tau_column = tau_column, tau_stats_column = None, 
                        pressure_column = pressure_column, temperature_column = temp_column, 
                        nominal_temperature = 296, abundance_ratio_MI = {}, x_shift = 0.00)
spec_6 = Spectrum('260Torr-all-offsetcorr-06-goodPressure', 
                        concentration = { 7 :0.2093}, natural_abundance = True, diluent = 'air', 
                        etalons = {}, 
                        input_freq = True, frequency_column = freq_column,
                        input_tau = True, tau_column = tau_column, tau_stats_column = None, 
                        pressure_column = pressure_column, temperature_column = temp_column, 
                        nominal_temperature = 296, abundance_ratio_MI = {}, x_shift = 0.00)
spec_7 = Spectrum('260Torr-all-offsetcorr-07-goodPressure', 
                        concentration = { 7 : 0.2093}, natural_abundance = True, diluent = 'air', 
                        etalons = {}, 
                        input_freq = True, frequency_column = freq_column,
                        input_tau = True, tau_column = tau_column, tau_stats_column = None, 
                        pressure_column = pressure_column, temperature_column = temp_column, 
                        nominal_temperature = 296, abundance_ratio_MI = {}, x_shift = 0.00)
spec_8 = Spectrum('260Torr-all-offsetcorr-08-goodPressure', 
                        concentration = { 7 :0.2093}, natural_abundance = True, diluent = 'air', 
                        etalons = {}, 
                        input_freq = True, frequency_column = freq_column,
                        input_tau = True, tau_column = tau_column, tau_stats_column = None, 
                        pressure_column = pressure_column, temperature_column = temp_column, 
                        nominal_temperature = 296, abundance_ratio_MI = {}, x_shift = 0.00)

#Add all spectrum to a Dataset object


SPECTRA = Dataset([spec_1, spec_2, spec_3, spec_4, spec_5, spec_6, spec_7, spec_8], 'Old Data', baseline_order = order_baseline_fit)

#Generate Baseline Parameter list based on number of etalons in spectra definitions and baseline order
BASE_LINELIST = SPECTRA.generate_baseline_paramlist()

#Read in Possible linelists

PARAM_LINELIST = pd.read_csv('Singlet Delta_Mendonca.csv')



In [3]:
FITPARAMS = Generate_FitParam_File(SPECTRA, PARAM_LINELIST, BASE_LINELIST, lineprofile = 'SDVP', linemixing = False, 
                                  fit_intensity = Fit_Intensity, threshold_intensity = IntensityThreshold, 
                                  nu_constrain = True, sw_constrain = True, gamma0_constrain = True, delta0_constrain = True, 
                                   aw_constrain = True, as_constrain = True, 
                                   nuVC_constrain = True, eta_constrain =True, linemixing_constrain = True)

FITPARAMS.generate_fit_param_linelist_from_linelist(vary_nu = {7:{1:False, 2:False, 3:False}}, vary_sw = {7:{1:False, 2:False, 3:False}},
                                                    vary_gamma0 = {7:{1: False, 2:False, 3: False}, 1:{1:False}}, vary_n_gamma0 = {7:{1:False}}, 
                                                    vary_delta0 = {7:{1: False, 2:False, 3: False}, 1:{1:False}}, vary_n_delta0 = {7:{1:False}}, 
                                                    vary_aw = {7:{1: False, 2:False, 3: False}, 1:{1:False}}, vary_n_gamma2 = {7:{1:False}}, 
                                                    vary_as = {7:{1:False}}, vary_n_delta2 = {7:{1:False}}, 
                                                    vary_nuVC = {7:{1:False}}, vary_n_nuVC = {7:{1:False}},
                                                    vary_eta = {}, vary_linemixing = {7:{1:False}})

FITPARAMS.generate_fit_baseline_linelist(vary_baseline = True, vary_concentration = {7:False, 1:False}, vary_xshift = False, 
                                      vary_etalon_amp= False, vary_etalon_freq= False, vary_etalon_phase= False)

,Concentration_O2,Concentration_O2_err,Concentration_O2_vary,baseline_a,baseline_a_err,baseline_a_vary,baseline_b,baseline_b_err,baseline_b_vary,baseline_c,baseline_c_err,baseline_c_vary,x_shift,x_shift_err,x_shift_vary
Spectrum Number,,,,,,,,,,,,,,,
1.0,0.2093,0,False,0.866245,0,True,0.0,0,True,0.0,0,True,0.0,0,False
2.0,0.2093,0,False,0.841745,0,True,0.0,0,True,0.0,0,True,0.0,0,False
3.0,0.2093,0,False,0.852982,0,True,0.0,0,True,0.0,0,True,0.0,0,False
4.0,0.2093,0,False,0.866125,0,True,0.0,0,True,0.0,0,True,0.0,0,False
5.0,0.2093,0,False,0.867294,0,True,0.0,0,True,0.0,0,True,0.0,0,False
6.0,0.2093,0,False,0.871162,0,True,0.0,0,True,0.0,0,True,0.0,0,False
7.0,0.2093,0,False,0.866970,0,True,0.0,0,True,0.0,0,True,0.0,0,False
8.0,0.2093,0,False,0.864159,0,True,0.0,0,True,0.0,0,True,0.0,0,False


In [ ]:
edit_files = Edit_Fit_Param_Files('Baseline_LineList', 'Parameter_LineList')
edit_baseline = edit_files.edit_generated_baselist()

edit_baseline


In [ ]:
edit_params = edit_files.edit_generated_paramlist()
edit_params

In [ ]:
edit_files.save_editted_baselist(edit_baseline)
edit_files.save_editted_paramlist(edit_params)
print ('')

In [ ]:
os.chdir(path)   
fit_data = Fit_DataSet(SPECTRA,'Baseline_LineList', 'Parameter_LineList', fit_intensity = Fit_Intensity, 
                baseline_limit = False, baseline_limit_factor = 10, 
                concentration_limit = False, concentration_limit_factor = 1.1, 
                etalon_limit = False, etalon_limit_factor = 2, #phase is constrained to +/- 2pi, 
                x_shift_limit = True, x_shift_limit_magnitude = 0.1, 
                nu_limit = True, nu_limit_magnitude = 0.5, 
                sw_limit = True, sw_limit_factor = 5, 
                gamma0_limit = True, gamma0_limit_factor = 10, n_gamma0_limit= False, n_gamma0_limit_factor = 50, 
                delta0_limit = True, delta0_limit_factor = 10, n_delta0_limit = False, n_delta0_limit_factor = 50, 
                SD_gamma_limit = True, SD_gamma_limit_factor = 10, n_gamma2_limit = False, n_gamma2_limit_factor = 50, 
                SD_delta_limit = False, SD_delta_limit_factor = 2, n_delta2_limit = False, n_delta2_limit_factor = 50, 
                nuVC_limit = False, nuVC_limit_factor = 2, n_nuVC_limit = False, n_nuVC_limit_factor = 50, 
                eta_limit = False, eta_limit_factor = 50, linemixing_limit = False, linemixing_limit_factor = 50)

params = fit_data.generate_params()


%prun result = fit_data.fit_data(params, wing_cutoff = 10, wing_wavenumbers = 1)
print (result.params.pretty_print())

fit_data.residual_analysis(result, indv_resid_plot=True)
fit_data.update_params(result)




In [ ]:
# The fit might be able to be sped up by getting rid of some of the for param calls in the definition